In [1]:
from glob import glob 
import os
from solucion_omar.paper2 import read_etsp_file, filter_arcs_by_time_window, hybrid_sa_ts

In [10]:

pathsProblems = sorted(glob(os.path.join('Dataset', 'Problems', 'Problems', 'ETSPTW-MCR', 'G-E5-MCR(30%)', '*.txt')))
pathsSolution = sorted(glob(os.path.join('Dataset', 'Solutions', 'Solutions', 'ETSPTW-MCR', 'G-E5-MCR(30%)', '*.txt')))

In [12]:
pathsSolution

['Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w120s5_1_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w120s5_2_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w120s5_3_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w120s5_4_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w120s5_5_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w140s5_1_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w140s5_2_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w140s5_3_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w140s5_4_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w140s5_5_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ETSPTW-MCR/G-E5-MCR(30%)/route_n20w160s5_1_MCR(30%).txt',
 'Dataset/Solutions/Solutions/ET

In [11]:
def get_solution_cost(path):
    with open(path) as file:
        lines = file.readlines()
        return int(lines[1])
get_solution_cost(pathsSolution[0])

271

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        problemPath = self.data[idx]
        optimalCost = get_solution_cost(self.labels[idx])

        data = read_etsp_file(problemPath)

        filtered_matrix, last_node = filter_arcs_by_time_window(data['nodes'], data['distance_matrix'])
        best_solution, best_cost, cost_progression, best_cost_progression = hybrid_sa_ts(
        nodes=data['nodes'],
        distance_matrix=data['distance_matrix'],
        num_customers=data['num_customers'],
        battery_capacity=data['battery_capacity'],
        energy_rate=data['energy_rate'],
        max_iterations=15000,     
        T0=10000,                 
        alpha=0.95,               
        cooling_interval=100,     
        perturbation_interval=500, 
        R=30,                     
        tabu_tenure_moves=400,     
        tabu_tenure_stations=75)

        gap = (best_cost - optimalCost)/optimalCost * 100

        return problemPath, optimalCost, best_cost, gap
    
    

In [ ]:
dataset = MyDataset(pathsProblems, pathsSolution)
dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=25,
    pin_memory=False,
    drop_last=False
)

In [23]:
import numpy as np
from tqdm import tqdm

batchSize = dataloader.batch_size
results = np.full((len(dataset), 3), fill_value=np.nan, dtype=np.float32)

dataForDf = {
    'file_path':[],
    'optimal_cost':[], 
    'best_cost':[], 
    'gap':[]
}
i=0
for batch in tqdm(dataloader):
    file_path, optimal_cost, best_cost, gap =  batch
    print(batch)

    dataForDf['file_path'].append(file_path[0])
    dataForDf['optimal_cost'].append(optimal_cost.item())
    dataForDf['best_cost'].append(best_cost.item())
    dataForDf['gap'].append(gap.item())

    i+=1


  4%|▍         | 1/25 [00:30<12:11, 30.49s/it]

[('Dataset/Problems/Problems/ETSPTW-MCR/G-E5-MCR(30%)/n20w120s5_1_MCR(30%).txt',), tensor([271]), tensor([372.6667], dtype=torch.float64), tensor([37.5154], dtype=torch.float64)]


  8%|▊         | 2/25 [00:57<10:52, 28.35s/it]

[('Dataset/Problems/Problems/ETSPTW-MCR/G-E5-MCR(30%)/n20w120s5_2_MCR(30%).txt',), tensor([233]), tensor([261.6667], dtype=torch.float64), tensor([12.3033], dtype=torch.float64)]
[('Dataset/Problems/Problems/ETSPTW-MCR/G-E5-MCR(30%)/n20w120s5_3_MCR(30%).txt',), tensor([317]), tensor([362.3333], dtype=torch.float64), tensor([14.3007], dtype=torch.float64)]
[('Dataset/Problems/Problems/ETSPTW-MCR/G-E5-MCR(30%)/n20w120s5_4_MCR(30%).txt',), tensor([314]), tensor([376.6667], dtype=torch.float64), tensor([19.9575], dtype=torch.float64)]
[('Dataset/Problems/Problems/ETSPTW-MCR/G-E5-MCR(30%)/n20w120s5_5_MCR(30%).txt',), tensor([249]), tensor([331.3333], dtype=torch.float64), tensor([33.0656], dtype=torch.float64)]
[('Dataset/Problems/Problems/ETSPTW-MCR/G-E5-MCR(30%)/n20w140s5_1_MCR(30%).txt',), tensor([181]), tensor([249.], dtype=torch.float64), tensor([37.5691], dtype=torch.float64)]
[('Dataset/Problems/Problems/ETSPTW-MCR/G-E5-MCR(30%)/n20w140s5_2_MCR(30%).txt',), tensor([279]), tensor([320

 40%|████      | 10/25 [07:37<11:26, 45.74s/it]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
results = pd.DataFrame(dataForDf)
results

In [ ]:
results.to_csv("results G-E5-MCR(30%)", index=False)